In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.estimators import PC, HillClimbSearch, MmhcEstimator
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.preprocessing import KBinsDiscretizer
import time
import warnings

# Игнорирование возникающих предупреждений.
warnings.filterwarnings('ignore')

<h2>Подготовка данных</h2>

In [ ]:
vk_data = pd.read_csv('./data/vk_interests_finance.csv')
vk_data.head()

In [ ]:
# Оставление необходимых столбцов.
vk_data = vk_data[['sex', 'relation', 'is_parent', 'has_pets', 'age', 'mean_tr', 'median_tr', 'tr_per_month']]

# Получение информации о датасете.
vk_data.info()

In [ ]:
vk_data.head()

In [ ]:
# Разделение выборки на тренировочную и тестовую.
x_train, x_test = sklearn.model_selection.train_test_split(vk_data, test_size=0.2, random_state=42)

In [ ]:
# Дискретизация непрерывных значений.
trainsformed_data = x_train.copy()
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
data_discrete = est.fit_transform(trainsformed_data.values[:,4:8])
trainsformed_data[['age', 'mean_tr',	'median_tr', 'tr_per_month']] = data_discrete

In [ ]:
# Вывод итоговой тренировочной выборки.
trainsformed_data.head()

In [ ]:
# Дискретизация тестовой выборки.
testformed_data = x_test.copy()
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
data_discrete = est.fit_transform(x_test.values[:,4:8])
testformed_data[['age', 'mean_tr',	'median_tr', 'tr_per_month']] = data_discrete

In [ ]:
# Итоговый вид тестовой выборки.
testformed_data.head()

<h2>Построение корреляционной матрицы</h2>

In [ ]:
#отбор столбцов для построения корреляционной матрицы
x_matrix = trainsformed_data

#построение диагональной маски
mask = np.zeros_like(x_matrix.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(x_matrix.corr(), mask = mask, annot=True, fmt= '.1f', ax = ax, cmap = 'Blues')

<h2>Hill Climb Search</h2>

In [ ]:
hc = HillClimbSearch(trainsformed_data)

<h2>PC algorithm</h2>

In [ ]:
pc = PC(trainsformed_data)

In [ ]:
# Создание списка оценок для hill-climbing.
scores = ['k2score', 'bdeuscore', 'bicscore']

# Создание списка методов структурного обучения.
methods = ['HC + K2', 'HC + BDeu', 'HC + BIC', 'PC']

# Создание списка моделей.
models = []

<h3>Получения трех разных моделей путем смешения hill-climbing и трех методов оценок + моделей PC алгоритма</h3>

In [ ]:
for score in scores:
    tic = time.time()
    models.append(hc.estimate(scoring_method=score))
    print('\n\n HC + Оценка: ',  score, '. Время выполнения: ', time.time()-tic)

<h2>Проверка построенных структур данных</h2>

In [ ]:
G_K = nx.DiGraph()
G_K.add_edges_from(models[0].edges())
pos = nx.layout.circular_layout(G_K)
nx.draw(G_K, pos, node_size=10, font_size=10, with_labels=True, font_weight='bold')

In [ ]:
G_K = nx.DiGraph()
G_K.add_edges_from(models[1].edges())
pos = nx.layout.circular_layout(G_K)
nx.draw(G_K, pos, node_size=10, font_size=10, with_labels=True, font_weight='bold')

In [ ]:
G_K = nx.DiGraph()
G_K.add_edges_from(models[2].edges())
pos = nx.layout.circular_layout(G_K)
nx.draw(G_K, pos, node_size=10, font_size=10, with_labels=True, font_weight='bold')

<h3>Создание моделей байесовских сетей</h3>

In [ ]:
# Создание списка байесовских сетей.
bns = []

for i in range(len(models)):
    bns.append(BayesianModel(models[i].edges()))

<h3>Обучение</h3>

In [ ]:
for i in range(len(bns)):
    tic = time.time()
    bns[i].fit(trainsformed_data)
    print('\n\n Hill-climb + Оценка: ',  scores[i], '. Время выполнения: ', time.time()-tic)

<h2>Тестирование</h2>

In [ ]:
i = 0
times = []

# Запуск цикла для удаления и восстановления каждого параметра.
for column in testformed_data.columns:
    mis_data = testformed_data.drop(columns=[column])

    print('\n Потерян: ', column, '\n')

    for j in range(len(bns)):
        # Заполнение пропусков в сетях.
        tic = time.time()
        result_data = bns[j].predict(mis_data)
        print('\n\n', methods[j], 'Время выполнения: ', time.time()-tic)
        times.append(time.time()-tic)

        result_data.reset_index(drop=True, inplace=True)
        testformed_data.reset_index(drop=True, inplace=True)

        # Проверка типа утеряного параметра.
        if vk_data[column].dtypes == 'object':
            regime = le[i].inverse_transform(result_data.values.astype(int))
                
            # Сравнение категориальных признаков.
            print('\n Accuracy: ', accuracy_score(x_test[column], regime), '\n')

        else:
            parameters = ['age', 'mean_tr',	'median_tr', 'tr_per_month']
            if column in parameters:
                concat_data = testformed_data[parameters]
                concat_data[column] = result_data[column]

                porosity = est.inverse_transform(concat_data)
                check_data = pd.DataFrame(data=porosity, columns=parameters)
                
            else:
                check_data = result_data

        # Подсчет среднеквадратической ошибки.
        print('\n Среднеквадратическая ошибка: ', mean_squared_error(check_data[column], x_test[column], squared=False), '\n')
    
    if vk_data[column].dtypes == 'object':
        i += 1

In [ ]:
times_K2 = []
times_BDeu = []
times_BIC = []

for i in range(0,len(times)-2,3):
    times_K2.append(times[i])
    times_BDeu.append(times[i+1])
    times_BIC.append(times[i+2])

In [ ]:
print("\n Среднее время предсказания HC+K2: ", np.mean(times_K2), "\n")
print("\n Среднее время предсказания HC+BDeu: ",np.mean(times_BDeu), "\n")
print("\n Среднее время предсказания HC+BIC: ",np.mean(times_BIC), "\n")